# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 21, 7, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            print(dt.hour, dt.minute)
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = True

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [20, 25, 30, 35]},
              'STOCH': {'fastk_period': [3, 5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [2, 3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Number of combinations is 4800


  0%|                                                                                                                                                                                                 | 0/4800 [00:00<?, ?it/s]


Load the datasets...
Binance
ByBit
BinanceFutures
ByBitPerpetual


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4800/4800 [2:44:57<00:00,  2.06s/it]


# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  5                  10                            85.144583   
               25            7                  3                  5                  10                            89.201389   
               35            11                 3                  9                  20                            66.518698   
12             20            7                  3                  5                  10                            87.384410   
16             30            5                  4                  3                  10                            74.307396   
               25            9                  3                  5                  10                            74.705347   
12             20            7                  4                  5                  10                            62.384340   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.652014   
14             30            5                  4                  5                  10                            73.236944   
16             25            7                  3                  7                  10                            60.069444   
12             20            9                  3                  5                  10                            75.434340   
                             7                  3                  7                  10                            68.402778   
16             25            7                  4                  5                  10                            57.141528   
                             5                  3                  7                  10                            61.805556   
12             20            5                  3                  7                  10                            61.805556   
14             20            9                  3                  7                  10                            55.266319   
               25            9                  3                  5                  10                            72.130694   
16             25            5                  4                  5                  10                            86.284722   
                             3                  3                  5                  20                            70.886620   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  5                  10                         1.423924   
               25            7                  3                  5                  10                         1.795312   
               35            11                 3                  9                  20                         0.455156   
12             20            7                  3                  5                  10                         1.720382   
16             30            5                  4                  3                  10                         0.796840   
               25            9                  3                  5                  10                         0.902917   
12             20            7                  4                  5                  10                         0.559063   
               30            11                 3                  9                  25                         0.418698   
              

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [16], 'low_bound': [25]},
                  'STOCH': {'fastk_period': [7], 'slowk_period': [3],
                            'slowd_period': [5], 'low_bound': [10]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

In [5]:
total_stat.loc[(16, 35, 11, 3, 9, 25)]

pct_right_forecast_avg     66.328698
pct_price_diff_avg          0.451719
forecast_rank              15.618906
price_rank                 24.975677
forecasts_num             365.000000
Name: (16, 35, 11, 3, 9, 25), dtype: float64